# Imports (compatible for Python 2 and Python 3)

In [1]:
try:
    import urllib2 as ulib
except ImportError:
    import urllib3 as ulib

try:
    from StringIO import BytesIO as io
except ImportError:
    from io import BytesIO as io

try:
    from urllib import urlencode as urlencode
except ImportError:
    from urllib.parse import urlencode as urlencode
    
try:
    from urllib import urlopen as urlopen
except ImportError:
    from urllib.request import urlopen as urlopen
    
try:
    from urllib2 import HTTPError as HTTPError
except ImportError:
    from urllib.error import HTTPError as HTTPError

import ast
import pandas as pd

In [2]:
API_URL = 'https://fb55.carto.com:443/api/v2/sql?q='

def queryCartoDB(query, formatting = 'CSV', source = API_URL):
    '''queries carto datasets from a given carto account
    Arguments: 
    query - string: a valid sql query string
    format - outlut format  OPTIONAL (default CSV)
    source - a valid sql api endpoint OPTIONAL (default carto fb55 account)
    Returns:
    the return of the sql query AS A STRING
    NOTES:
    designed for the carto API, tested only with CSV return format'''
    
    
    data = urlencode({'format': formatting, 'q': query}).encode("utf-8")
    try:
        response = urlopen(source, data)
        return response.read()
    except HTTPError as e:
        raise (ValueError('\n'.join(ast.literal_eval(e.readline())['error'])))
        
def get_data(query):
    try:
        return pd.read_csv(io(queryCartoDB(query)), sep = ',')
    except ValueError as v:
        print (str(v))

### Task 1

In [3]:
query = '''SELECT * FROM fb55.citibike; '''
table = get_data(query)
table.head(5)

,the_geom,start_station_name,the_geom_webmercator,field_1,start_station_longitude,tripduration,starttime,stoptime,start_station_id,cartodb_id,start_station_latitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bikeid,usertype,birth_year,gender
0,NaN,E 20 St & Park Ave,NaN,175,-73.987520,1090,2015-02-01 01:23:00+00,2015-02-01 01:42:00+00,503,107,40.738274,229,Great Jones St,40.727434,-73.993790,19718,Subscriber,1961.0,1
1,NaN,W 43 St & 10 Ave,NaN,1159,-73.994618,682,2015-02-01 10:55:00+00,2015-02-01 11:07:00+00,515,1088,40.760094,490,8 Ave & W 33 St,40.751551,-73.993934,21501,Subscriber,1981.0,1
2,NaN,E 6 St & Avenue B,NaN,2827,-73.981854,751,2015-02-01 13:59:00+00,2015-02-01 14:11:00+00,317,2759,40.724537,466,W 25 St & 6 Ave,40.743954,-73.991449,14788,Subscriber,1990.0,1
3,NaN,E 12 St & 3 Ave,NaN,4961,-73.988900,272,2015-02-01 17:28:00+00,2015-02-01 17:32:00+00,483,4893,40.732233,345,W 13 St & 6 Ave,40.736494,-73.997044,16219,Subscriber,1961.0,1
4,NaN,W 41 St & 8 Ave,NaN,6156,-73.990026,240,2015-02-01 21:36:00+00,2015-02-01 21:40:00+00,477,6090,40.756405,490,8 Ave & W 33 St,40.751551,-73.993934,18266,Customer,NaN,0


In [4]:
t1 = '''SELECT start_station_id, tripduration FROM fb55.citibike WHERE tripduration <= 3*60;'''
table1 = get_data(t1)
table1.head(n=10)

,start_station_id,tripduration
0,492,159
1,545,180
2,116,120
3,350,144
4,317,159
5,195,89
6,279,171
7,345,164
8,237,177
9,325,155


In [5]:
table1.tail(n=10)

,start_station_id,tripduration
1791,223,138
1792,127,139
1793,442,106
1794,330,151
1795,127,148
1796,127,148
1797,151,81
1798,489,172
1799,265,113
1800,350,127


In [6]:
t2='''SELECT DISTINCT start_station_id FROM fb55.citibike;'''
table2 = get_data(t2)
table2

,start_station_id,Unnamed: 1
0,120,NaN
1,285,NaN
2,251,NaN
3,195,NaN
4,453,NaN
5,144,NaN
6,266,NaN
7,263,NaN
8,216,NaN
9,328,NaN


In [7]:
t3='''SELECT COUNT(tripduration), AVG(tripduration),MIN(tripduration), MAX(tripduration) FROM fb55.citibike ;'''
table3 = get_data(t3)
table3

,count,avg,min,max
0,46200,675.865823,60,43016


### Task2

In [8]:
t4='''SELECT tripduration FROM fb55.citibike WHERE starttime BETWEEN '2015-02-01 00:00:00+00' AND '2015-02-02 00:00:00+00';'''
table4=get_data(t4)
table4.head(n=10)

,tripduration,Unnamed: 1
0,1090,NaN
1,682,NaN
2,751,NaN
3,272,NaN
4,240,NaN
5,447,NaN
6,277,NaN
7,526,NaN
8,502,NaN
9,943,NaN


In [29]:
t5='''SELECT tripduration, date_part('dow', starttime) AS dow FROM fb55.citibike WHERE DATE_PART('dow', starttime) BETWEEN 5 AND 7;'''
table5=get_data(t5)
table5

,tripduration,dow
0,409,5
1,561,5
2,448,5
3,497,5
4,932,5
5,610,5
6,575,5
7,511,5
8,315,5
9,657,5


In [38]:
t6='''SELECT AVG(tripduration) FROM fb55.citibike WHERE DATE_PART('dow', starttime) BETWEEN 5 AND 7;'''
table6=get_data(t6)
table6

,avg,Unnamed: 1
0,658.922681,NaN


In [43]:
t7='''SELECT tripduration, date_part('dow', starttime) AS dow FROM fb55.citibike WHERE DATE_PART('dow', starttime) <5;'''
table7=get_data(t7)
table7

,tripduration,dow
0,1090,0
1,682,0
2,751,0
3,272,0
4,240,0
5,376,1
6,1217,1
7,441,2
8,1475,2
9,420,2


In [44]:
t8='''SELECT AVG(tripduration) FROM fb55.citibike WHERE DATE_PART('dow', starttime) <5;'''
table8=get_data(t8)
table8

,avg,Unnamed: 1
0,684.308545,NaN


### Task 3

In [47]:
t9='''SELECT start_station_name, COUNT(start_station_name) FROM fb55.citibike GROUP BY start_station_name;'''
table9=get_data(t9)
table9

,start_station_name,count
0,E 30 St & Park Ave S,227
1,Pearl St & Anchorage Pl,54
2,W 56 St & 6 Ave,164
3,West Thames St,107
4,Bank St & Washington St,125
5,8 Ave & W 52 St,133
6,E 11 St & Broadway,115
7,DeKalb Ave & Skillman St,10
8,Atlantic Ave & Fort Greene Pl,36
9,Avenue D & E 8 St,38


### Task 4